# เอเจนต์ Azure AI พร้อมการสนับสนุน Model Context Protocol (MCP)

สมุดบันทึกนี้แสดงวิธีการใช้งานเอเจนต์ Azure AI ร่วมกับเครื่องมือ Model Context Protocol (MCP) เพื่อสร้างเอเจนต์อัจฉริยะที่สามารถใช้เซิร์ฟเวอร์ MCP ภายนอกเพื่อเพิ่มความสามารถ


## ติดตั้งแพ็กเกจ NuGet ที่จำเป็น

ก่อนอื่น เราต้องติดตั้งแพ็กเกจ Azure AI Agents Persistent ซึ่งเป็นแพ็กเกจหลักที่ให้ฟังก์ชันการทำงานสำหรับการใช้งาน Azure AI Agents


## ข้อดีของการยืนยันตัวตนแบบไม่ใช้คีย์

สมุดบันทึกนี้แสดงให้เห็นถึง **การยืนยันตัวตนแบบไม่ใช้คีย์** ซึ่งมีข้อดีหลายประการ:
- ✅ **ไม่ต้องจัดการ API keys** - ใช้การยืนยันตัวตนที่อิงกับ Azure
- ✅ **เพิ่มความปลอดภัย** - ไม่มีการเก็บข้อมูลลับในโค้ดหรือการตั้งค่า
- ✅ **การหมุนเวียนข้อมูลรับรองอัตโนมัติ** - Azure จัดการวงจรชีวิตของข้อมูลรับรอง
- ✅ **การเข้าถึงตามบทบาท** - ใช้ Azure RBAC เพื่อกำหนดสิทธิ์อย่างละเอียด

`DefaultAzureCredential` จะเลือกใช้แหล่งข้อมูลรับรองที่ดีที่สุดโดยอัตโนมัติ:
1. Managed Identity (เมื่อรันใน Azure)
2. ข้อมูลรับรองจาก Azure CLI (ระหว่างการพัฒนา)
3. ข้อมูลรับรองจาก Visual Studio
4. ตัวแปรสภาพแวดล้อม (ถ้ามีการตั้งค่า)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## นำเข้าพื้นที่ชื่อที่จำเป็น

นำเข้าพื้นที่ชื่อที่จำเป็นสำหรับ Azure AI Agents และ Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## การตั้งค่า Azure AI Agent Client (การยืนยันตัวตนแบบไม่ใช้คีย์)

ตั้งค่าตัวแปรการกำหนดค่าและสร้าง PersistentAgentsClient โดยใช้ **การยืนยันตัวตนแบบไม่ใช้คีย์**:
- **projectEndpoint**: จุดเชื่อมต่อของโครงการ Azure AI Foundry
- **modelDeploymentName**: ชื่อของโมเดล AI ที่ถูกปรับใช้ (GPT-4.1 nano)
- **mcpServerUrl**: URL ของเซิร์ฟเวอร์ MCP (Microsoft Learn API)
- **mcpServerLabel**: ป้ายกำกับเพื่อระบุเซิร์ฟเวอร์ MCP
- **DefaultAzureCredential**: ใช้ managed identity, Azure CLI หรือแหล่งข้อมูลรับรองอื่น ๆ (ไม่ต้องใช้ API keys)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## สร้างคำจำกัดความเครื่องมือ MCP

สร้างคำจำกัดความเครื่องมือ MCP ที่เชื่อมต่อกับเซิร์ฟเวอร์ Microsoft Learn MCP สิ่งนี้จะช่วยให้ตัวแทนสามารถเข้าถึงเนื้อหาและเอกสารของ Microsoft Learn ได้


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## สร้าง AI Agent

สร้าง AI agent ที่มีความสามารถในการทำงานอย่างต่อเนื่องด้วยโมเดลและเครื่องมือ MCP ที่กำหนดไว้ ตัว agent ถูกตั้งค่าด้วย:
- โมเดล GPT-4.1 nano
- คำแนะนำในการใช้เครื่องมือ MCP เพื่อช่วยเหลือ
- การเข้าถึงเซิร์ฟเวอร์ Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## สร้างเธรดและส่งข้อความ

สร้างเธรดการสนทนาและส่งข้อความถึงผู้ใช้เพื่อสอบถามเกี่ยวกับความแตกต่างระหว่าง Azure OpenAI และ OpenAI สิ่งนี้จะเป็นการทดสอบความสามารถของตัวแทนในการใช้เครื่องมือ MCP เพื่อให้ข้อมูลที่ถูกต้อง


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## การตั้งค่าทรัพยากร MCP Tool (แบบไม่ใช้คีย์)

ตั้งค่าทรัพยากรสำหรับ MCP Tool สำหรับการใช้งานแบบไม่ใช้คีย์อย่างแท้จริง คุณสามารถลบ custom headers ได้หากเซิร์ฟเวอร์ MCP รองรับการยืนยันตัวตนโดยใช้ Azure identity ตัวอย่างด้านล่างแสดงวิธีการเพิ่ม headers หากจำเป็น แต่สำหรับกรณีที่ไม่ใช้คีย์ อาจไม่ต้องใช้ headers เหล่านี้


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## เริ่มการทำงานของ Agent

สร้างและเริ่มการทำงานเพื่อประมวลผลข้อความของผู้ใช้ Agent จะใช้เครื่องมือและทรัพยากร MCP ที่ตั้งค่าไว้เพื่อสร้างคำตอบ


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## ตรวจสอบการทำงานและจัดการการอนุมัติเครื่องมือ (แบบไม่ใช้คีย์)

ตรวจสอบสถานะการทำงานของเอเจนต์และจัดการการอนุมัติเครื่องมือที่จำเป็น วนลูปนี้:
1. รอให้การทำงานเสร็จสิ้นหรือมีการดำเนินการที่ต้องการ
2. อนุมัติการเรียกใช้เครื่องมือ MCP โดยอัตโนมัติเมื่อจำเป็น
3. สำหรับการยืนยันตัวตนแบบไม่ใช้คีย์ อาจไม่จำเป็นต้องใช้ headers หากเซิร์ฟเวอร์ MCP รองรับ Azure identity


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## แสดงผลลัพธ์การสนทนา

ดึงและแสดงข้อความทั้งหมดในเธรด โดยแสดงทั้งคำถามของผู้ใช้และคำตอบของตัวแทน ข้อความจะแสดงตามลำดับเวลา พร้อมตัวบ่งชี้เวลาและบทบาท


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
